In [379]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [380]:
import xml.etree.ElementTree as etree
source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

In [381]:
def format_variables(model):
    variables       = model.find(source_xmlns + 'variables')
    stocks          = variables.findall(source_xmlns + 'stock')
    auxs_and_ctes   = variables.findall(source_xmlns + 'aux')
    flows           = variables.findall(source_xmlns + 'flow')
    dependencies    = variables.find(source_xmlns_isee + 'dependencies')
    dependencies    = dependencies.findall(source_xmlns + 'var') if dependencies is not None else [] 
    dependencies_ = {}
    def get_params(eqn):
        import re
        text = eqn.replace(' ', '').replace('.','&')
        res = filter(lambda x : x != '', list(map(lambda x : x.replace('&', '.'), re.split("[()-/*]+", text))))
        return res

    for dep in dependencies:
        name = dep.get('name')
        inputs = dep.findall(source_xmlns + 'in')
        inputs_ = []
        for in_dep in inputs:
            inputs_.append(in_dep.text)
        dependencies_.update({name : inputs_})

    auxs_ = {}
    vars_with_dependencies = dependencies_.keys()
    for var in auxs_and_ctes:
        name = var.get('name')
        eqn  = var.find(source_xmlns + 'eqn').text
        if name in vars_with_dependencies:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})
        else:
            auxs_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})

    stocks_ = {}
    for stock in stocks:
        name = stock.get('name')
        eqn  = stock.find(source_xmlns + 'eqn').text
        inflows  = list(map(lambda x : x.text, stock.findall(source_xmlns + 'inflow')))
        outflows = list(map(lambda x : x.text, stock.findall(source_xmlns + 'outflow')))
        non_negative = True if stock.find(source_xmlns + 'non_negative') is not None else False
        stocks_.update({name : {
            'eqn' : eqn, 
            'inflows' : inflows, 
            'outflows' : outflows,
            'non_negative' : non_negative
        }})

    flows_ = {}
    for flow in flows:
        name = flow.get('name')
        eqn  = flow.find(source_xmlns + 'eqn').text
        flows_.update({name : {'eqn' : eqn, 'eqn_params' : get_params(eqn)}})
        
    return {
        'dependencies' : dependencies_,
        'stocks' : stocks_,
        'flows' : flows_,
        'auxs'  : auxs_
    }

def format_variables_top_model(top_model):
    variables = top_model.find(source_xmlns + 'variables')
    modules = variables.findall(source_xmlns + 'module')
    auxs = variables.findall(source_xmlns + 'aux')
    modules_ = {}
    for module in modules:
        name = module.get('name')
        connections = module.findall(source_xmlns + 'connect')
        connections_ = []
        for connection in connections:
            connections_.append({'from' : connection.get('from'), 'to' : connection.get('to')})
        modules_.update({name : connections_})
    auxs_ = {}

    for aux in auxs:
        name = aux.get('name')
        eqn  = aux.find(source_xmlns + 'eqn').text
        auxs_.update({name : eqn})
    return {
        'modules' : modules_,
        'auxs' : auxs_
    }

# Goodwin

In [386]:
def generateDEVSML(formatted_variables):
    
    auxs = formatted_variables['auxs']
    flows = formatted_variables['flows']
    stocks= formatted_variables['stocks']
    dependencies = formatted_variables['dependencies']

    TEMPLATE_COUPLED_DEVSML = 'template-coupled-devsml.xml'

    all_variables = stocks.keys() + flows.keys() + auxs.keys()
    internal_connections = []
    atomics = []

    #####################
    # Atomicos
    for stockName, stockAttr in stocks.iteritems():
        atomics.append({
            'name' : stockName + 'Tot',
            'model' : 'Ftot',
            'in_ports' : [{'name' : elem + 'Plus' , 'type' : 'in_plus'} for elem in stockAttr['inflows']] +
                         [{'name' : elem + 'Minus', 'type' : 'in_minius'} for elem in stockAttr['outflows']],
            'out_ports' : [{'name' :'out', 'type' : 'out'}]
        })
        atomics.append({
            'name' : stockName,
            'model' : 'QSS1',
            'in_ports' : [{'name' : 'in', 'type' : 'in'}],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'x0', 'function' : stockAttr['eqn']},
                {'name' : 'dQRel', 'function' : 1e-4},
                {'name' : 'dQMin', 'function' : 1e-4}
            ]
        })
    for auxName, auxAttr in auxs.iteritems():
        eqn_params = filter(lambda x : x in all_variables, auxAttr['eqn_params'])

        atomics.append({
            'name' : auxName,
            'model' : 'Cte' if eqn_params == [] else 'Aux',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in eqn_params],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : auxAttr['eqn']}
            ]
        })

    for flowName, flowAttr in flows.iteritems():
        atomics.append({
            'name' : flowName + 'Plus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })
        atomics.append({
            'name' : flowName + 'Minus',
            'model' : 'Fpm',
            'in_ports' : [{'name' : elem, 'type' : 'in'} for elem in
                          filter(lambda x : x in all_variables, flowAttr['eqn_params'])],
            'out_ports' : [{'name' : 'out', 'type' : 'out'}],
            'parameters' : [
                {'name' : 'function', 'function' : flowAttr['eqn']}
            ]
        })

    #####################
    # Conexiones internas

    # Ftot => Integrador
    for stockName, stockAttr in stocks.iteritems():
        internal_connections.append({
            'component_from' : stockName + 'Tot',
            'component_to' : stockName,
            'port_from' : 'out',
            'port_to' : 'in'
        })
    # Fpm => Ftot
    for flowName, flowAttr in flows.iteritems():
        for stockName, stockAttr in stocks.iteritems():
            if flowName in stockAttr['inflows']:
                internal_connections.append({
                    'component_from' : flowName + 'Plus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Plus'
                })
            if flowName in stockAttr['outflows']:
                internal_connections.append({
                    'component_from' : flowName + 'Minus',
                    'component_to' : stockName + 'Tot',
                    'port_from' : 'out',
                    'port_to' : flowName + 'Minus'
                })
    # Stock => {Flow, Aux}
    for stockName, stockAttr in stocks.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if stockName in flowAttr['eqn_params'] and flowName in stockAttr['inflows']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
            if stockName in flowAttr['eqn_params'] and flowName in stockAttr['outflows']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : 'out',
                    'port_to' : stockName
                })
        # Aux
        for auxName, auxAttr in auxs.iteritems():
            if stockName in auxAttr['eqn_params']:
                internal_connections.append({
                    'component_from' : stockName,
                    'component_to' : auxName,
                    'port_from' : 'out',
                    'port_to' : stockName
                })
    # Aux => {Aux, Flow}
    for auxName, auxAttr in auxs.iteritems():
        # Flow
        for flowName, flowAttr in flows.iteritems():
            if auxName in flowAttr['eqn_params']:
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Plus',
                    'port_from' : 'out',
                    'port_to' : auxName
                })
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : flowName + 'Minus',
                    'port_from' : 'out',
                    'port_to' : auxName
                })
        # Aux
        for auxName2, auxAttr2 in auxs.iteritems():
            if auxName in auxAttr2['eqn_params'] and auxName != auxName2:
                internal_connections.append({
                    'component_from' : auxName,
                    'component_to' : auxName2,
                    'port_from' : 'out',
                    'port_to' : auxName
                })

    #####################
    # Input Ports : los Aux's que no tienen valor, o que tienen un valor numerico
    in_ports = []
    for auxName, auxAttr in auxs.iteritems():
        # NOTA : los que estan vacios ASUMO que es porque viene de un modelo acoplado de afuera
        emptyEqn = (auxAttr['eqn'] == '')
        if emptyEqn or all([not elem in all_variables for elem in auxAttr['eqn_params']]):
            in_ports.append(auxName)

    # Output Ports : en ppio., todos los stocks
    out_ports = []
    for stockName, stockAttr in stocks.iteritems():
        out_ports.append(stockName)

    context = {
        'coupled_model_name' : 'top',
        'in_ports' : in_ports,
        'out_ports' : out_ports,
        'atomics' : atomics,
        'internal_connections' : internal_connections,
        'external_input_connections' : [],
        'external_output_connections' : []
    }
    z = render_template(TEMPLATE_COUPLED_DEVSML, context)
    with open('goodwin/goodwin-devsml-traduccion.xml', 'w') as f:
        f.write(render_template(TEMPLATE_DEVSML,context))
        
    return True

# Traducciones

In [388]:
DIR_XMILE = 'goodwin/versions/goodwin-1.xmile'
parser = ET.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

header = root.find(source_xmlns + 'model')
sim_specs = root.find(source_xmlns + 'sim_specs')
prefs = root.find(source_xmlns_isee + 'prefs')
multiplayer_settings = root.find(source_xmlns_isee + 'multiplayer_settings')
model_units = root.find(source_xmlns + 'model_units')
model = root.find(source_xmlns + 'model')

formatted_variables = format_variables(model)
generateDEVSML(formatted_variables)

True

# Modelos XMILE acoplados

In [ ]:
DIR_XMILE = 'lotka_volterra/lotka_volterra_coupled.xmile'
parser = ET.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()

models = root.findall(source_xmlns + 'model')
top = None
modules = []
for model in models:
    name = model.get('name')
    if name is None:
        top = model
    else:
        modules.append(model)
variables_top = format_variables_top_model(top)
variables_modules = [format_variables(module) for module in modules]